In [3]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pdb
import requests
import re

import numpy as np
import pandas as pd
from functools import reduce
from collections import Counter

import networkx as nx

import signal

import warnings
warnings.filterwarnings("ignore")

from wiki_intro_scrapper import WikiIntroScrapper
from WikiMultiQuery import wiki_multi_query
from graph_helpers import create_dispersion_df, sort_dict_values, format_categories, compare_categories, rank_order

%aimport wiki_intro_scrapper
%aimport WikiMultiQuery
%aimport graph_helpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
class GraphCreator:

    def __init__(self, entry):
        self.graph = nx.DiGraph()

        self.entry = entry

        wis = WikiIntroScrapper(f"https://en.wikipedia.org/wiki/{entry}")
        wis.parse_intro_links()

        self.intro_nodes = wis.intro_link_titles

        self.visited = {entry}
        self.next_links = []
        
        self.categories = {}
        
        self.redirect_targets = []
        self.redirect_sources = {}
        
        self.query_articles([entry])

        # setup timeout function

        def handle_alarm(signum, frame):
            raise RuntimeError

        signal.signal(signal.SIGALRM, handle_alarm)

    def add_edges(self, articles):
        for article in articles:
            
            self.categories[article['title']] = format_categories([category.split("Category:")[1] for category in article['categories'] if not bool(re.findall(r"(articles)|(uses)|(commons)|(category\:use)", category, re.I))])
            
            self.graph.add_edges_from(
                [(article['title'], link) for link in article['links']])
            self.graph.add_edges_from(
                [(linkhere, article['title']) for linkhere in article['linkshere']])

    def update_edge_weights(self):
        for edge in self.graph.out_edges:
            weight = compare_categories(edge[0], edge[1], self.categories)
            self.graph.add_edge(edge[0], edge[1], weight=weight)
            
        for edge in self.graph.in_edges:
            weight = compare_categories(edge[0], edge[1], self.categories)
            self.graph.add_edge(edge[0], edge[1], weight=weight)
    
    def get_edge_weights(self):
        edge_weights = []
        for edge in self.graph.edges:
            edge_weights.append((edge[0], edge[1], self.graph.get_edge_data(edge[0], edge[1])['weight']))
        
#         for edge in self.graph.in_edges:
#             edge_weights.append((edge[0], edge[1], self.graph.get_edge_data(edge[0], edge[1])['weight']))
        
        return pd.DataFrame(edge_weights, columns=["source_node", "target_node", "edge_weight"]).sort_values("edge_weight", ascending=False).reset_index().drop("index", axis=1)
    
    def plot_graph(self):
        nx.draw(self.graph)
        plt.show()

    def get_shared_categories_with_source(self):
        cat_matches = {}
        for node in self.graph.nodes:
            cat_matches[node] = compare_categories(self.entry, node, self.categories, starting_count=0)
        return sort_dict_values(cat_matches, ['node', 'category_matches_with_source'], 'category_matches_with_source', ascending=False)
            
            
    def get_degrees(self):
        return sort_dict_values(dict(self.graph.degree()), ["node", "degree"], "degree",)

    def get_edges(self):
        in_edges = sort_dict_values(dict(Counter([edge[1] for edge in self.graph.in_edges()])), 
                            ['node', 'in_edges'], "in_edges")
        out_edges = sort_dict_values(dict(Counter([edge[0] for edge in self.graph.out_edges()])), 
                            ["node", 'out_edges'], 'out_edges')

        return in_edges.merge(out_edges, on="node")
    
    def get_centrality(self):
        return sort_dict_values(nx.eigenvector_centrality(self.graph, weight="weight"), ["node", "centrality"], "centrality")

    def get_dispersion(self, comparison_node=None, max_nodes=25_000):
        if not comparison_node:
            comparison_node = self.entry
            
        if max_nodes is None or len(self.graph.nodes) <= max_nodes:
            print("FULL")
            return sort_dict_values(nx.dispersion(self.graph, u=comparison_node), ['node', 'dispersion'], 'dispersion')
        else:
            print("EGO")
            # if the network is too large, perform calculation on ego graph of entry node
            ego = self.create_ego()
            return sort_dict_values(nx.dispersion(ego, u=comparison_node), ['node', 'dispersion'], 'dispersion')

    def get_pageranks(self):
        page_ranks = sorted([(key, value) for key, value in nx.algorithms.link_analysis.pagerank(
            self.graph, weight='weight').items()], key=lambda x: x[1], reverse=True)
        return pd.DataFrame(page_ranks, columns=["node", "page_rank"])

    def get_reciprocity(self):
        return sort_dict_values(nx.algorithms.reciprocity(self.graph, self.graph.nodes), ['node', 'reciprocity'], 'reciprocity')

    def get_adjusted_reciprocity(self):
        r = self.get_reciprocity()
        d = self.get_degrees()

        r_d = r.merge(d, on="node", how="inner")
        r_d['adjusted_reciprocity'] = r_d.reciprocity * r_d.degree

        adjusted_reci = r_d.sort_values("adjusted_reciprocity", ascending=False)
        return adjusted_reci.reset_index().drop(["degree", "reciprocity", "index"], axis=1)
    
    def get_shortes_path(self, source=None, ascending=False):
        if not source:
            source = self.entry
            
        paths = nx.algorithms.single_source_shortest_path_length(self.graph, source)
        return sort_dict_values(paths, ["node", "shortest_path_length_from_source"], "shortest_path_length_from_source", ascending=ascending)
    
    def get_dominator_counts(self, source=None):
        if not source:
            source = self.entry
            
        dom_dict = nx.algorithms.dominance.immediate_dominators(self.graph, start=source)
        
        dom_counts = {}

        for key, value in dom_dict.items():
            if value in dom_counts:
                dom_counts[value] += 1
            else:
                dom_counts[value] = 1
        for node in self.graph.nodes:
            if not node in dom_counts:
                dom_counts[node] = 0
        
        return sort_dict_values(dom_counts, ['node', 'immediate_dominator_count'], 'immediate_dominator_count')
    
    def get_hits(self):
        hits = nx.algorithms.link_analysis.hits_alg.hits(self.graph, max_iter=1000)
        return (sort_dict_values(hits[1], ['node', 'hits_authority'], 'hits_authority')
                .merge(sort_dict_values(hits[0], ['node', 'hits_hub'], 'hits_hub'), on="node"))
    
    def get_features_df(self, rank=False):
        dfs = []
        if rank:
            dfs.append(rank_order(self.get_degrees(), 'degree', ascending=False))
            dfs.append(rank_order(self.get_shared_categories_with_source(), 'category_matches_with_source', ascending=False))
            dfs.append(self.get_edges())
            dfs.append(rank_order(self.get_centrality(), 'centrality', ascending=True))
            dfs.append(rank_order(self.get_dispersion(), "dispersion", ascending=True))
            dfs.append(rank_order(self.get_pageranks(), "page_rank", ascending=False))
            dfs.append(rank_order(self.get_adjusted_reciprocity(), "adjusted_reciprocity", ascending=False))
            dfs.append(rank_order(self.get_shortes_path(), "shortest_path_length_from_source", ascending=True))
        
        else:
            dfs.append(self.get_degrees())
            dfs.append(self.get_shared_categories_with_source())
            dfs.append(self.get_edges())
            dfs.append(self.get_centrality())
            dfs.append(self.get_dispersion())
            dfs.append(self.get_pageranks())
            dfs.append(self.get_adjusted_reciprocity())
            dfs.append(self.get_shortes_path())
        
        return reduce(lambda left, right: pd.merge(left, right, on="node", how="outer"), dfs)
        
    
    def create_ego(self, node=None):
        if not node:
            node = self.entry

        ego = nx.ego_graph(self.graph, node)
        ego.name = node
        return ego

    def expand_network(self, group_size=10, timeout=10, log_progress=False):

        num_links = len(self.next_links)

        link_group = []

        for i in range(num_links):
            link = self.next_links.pop(0)
            if not link in self.visited:

                link_group.append(link)

                if len(link_group) == group_size or (i == num_links - 1 and len(link_group) > 0):
                    print("{:.2%}".format(i/num_links)) if log_progress else None
                    try:
                        signal.alarm(timeout)
                        self.visited.update(link_group)
                        self.query_articles(link_group)
                        signal.alarm(0)
                        link_group = []
                    except:
                        link_group = []
                        continue
        signal.alarm(0)

    def update_redirects(self, articles):
        for article in articles:
            if article.get("redirects"):
                self.redirect_targets.append(article["title"])
                for redirect in article["redirects"]:
                    self.redirect_sources[redirect] = len(self.redirect_targets) - 1
    
    def redraw_redirects(self):
        edges = list(self.graph.edges) # need this copy so 'edges' doesn't change size on iteration
        for edge in edges:
            if edge[0] in self.redirect_sources:
                self.graph.add_edge(self.redirect_targets[self.redirect_sources[edge[0]]], edge[1])
                
            if edge[1] in self.redirect_sources:
                self.graph.add_edge(edge[0], self.redirect_targets[self.redirect_sources[edge[1]]])
        
        self.remove_redirect_nodes()
    
    def remove_redirect_nodes(self):
        nodes = list(self.graph.nodes) # need this copy so 'nodes' doesn't change size on iteration
        for node in nodes:
            if node in self.redirect_sources:
                self.graph.remove_node(node)
    
    def update_next_links(self, articles):
        for article in articles:
            for link in article['links']:
                self.next_links.append(link)

    def query_articles(self, titles, generate_graph=True):
        articles = wiki_multi_query(titles)
        
        self.update_redirects(articles)
        
        self.update_next_links(articles)
        self.add_edges(articles)

## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [62]:
gc = GraphCreator("Orchestration")
print("Number of Links to Search:", len(gc.next_links))
print(gc.intro_nodes)

Number of Links to Search: 236
['Music', 'Orchestra', 'Musical ensemble', 'Concert band', 'Melody', 'Bassline', 'Classical music', 'Musical theatre', 'Film music', 'Arranger', 'Pit orchestra', 'Big band', 'Songwriter', 'Lead sheet', 'Rhythm section', 'Jazz guitar', 'Hammond organ']


2. We query all the nodes linked to from the entry point (expand our network one level for each node).

In [56]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect node. 

In [57]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [58]:
gc.update_edge_weights()

# Getting Our Feature Set

We have to options when generating our feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

In [59]:
features_df = gc.get_features_df(rank=False)
features_df

EGO


,node,degree,degree_ranked,category_matches_with_source,category_matches_with_source_ranked,in_edges,out_edges,centrality,centrality_ranked,dispersion,dispersion_ranked,page_rank,page_rank_ranked,adjusted_reciprocity,adjusted_reciprocity_ranked,shortest_path_length_from_source,shortest_path_length_from_source_ranked
0,Violin,11086,1,0,3,10644.0,442.0,6.562329e-02,3786,0.962963,64.0,0.014348,3,348.0,47,1.0,2.0
1,Trumpet,10699,2,0,3,10184.0,515.0,4.153617e-02,3764,2.108108,102.0,0.010171,6,296.0,55,1.0,2.0
2,Baroque,8203,3,0,3,7673.0,530.0,1.743578e-02,3647,0.250000,16.0,0.017191,1,406.0,34,1.0,2.0
3,Percussion instrument,8056,4,0,3,7785.0,271.0,2.362268e-02,3709,0.235294,14.0,0.011687,5,256.0,66,1.0,2.0
4,Music,7833,5,0,3,6667.0,1166.0,3.110630e-02,3744,4.846154,119.0,0.013968,4,558.0,21,1.0,2.0
5,Flute,7595,6,0,3,7106.0,489.0,4.069891e-02,3762,0.368421,22.0,0.008381,8,356.0,44,1.0,2.0
6,Trombone,6806,7,0,3,6403.0,403.0,3.882628e-02,3760,0.880000,52.0,0.005413,23,194.0,80,1.0,2.0
7,Java,6781,8,0,3,6387.0,394.0,7.681893e-03,3360,0.000000,1.0,0.016642,2,452.0,29,1.0,2.0
8,Cello,6435,9,1,2,5967.0,468.0,4.443777e-02,3771,2.400000,107.0,0.007263,14,380.0,38,1.0,2.0
9,Film score,6048,10,0,3,4816.0,1232.0,3.860807e-02,3759,8.666667,121.0,0.008206,9,640.0,15,1.0,2.0


In [64]:
def average_rank(row):
    return np.mean([
        row.degree_ranked,
#         row.centrality_ranked,
        row.dispersion_ranked,
#         row.page_rank_ranked,
#         row.adjusted_reciprocity_ranked,
    ]) * row.category_matches_with_source_ranked * row.shortest_path_length_from_source_ranked 

features_df["rank_average"] = features_df.apply(average_rank, axis=1)

features_df.sort_values("rank_average", ascending=True)

,node,degree,degree_ranked,category_matches_with_source,category_matches_with_source_ranked,in_edges,out_edges,centrality,centrality_ranked,dispersion,dispersion_ranked,page_rank,page_rank_ranked,adjusted_reciprocity,adjusted_reciprocity_ranked,shortest_path_length_from_source,shortest_path_length_from_source_ranked,rank_average
7,Java,6781,8,0,3,6387.0,394.0,7.681893e-03,3360,0.000000,1.0,0.016642,2,452.0,29,1.0,2.0,27.0
3,Percussion instrument,8056,4,0,3,7785.0,271.0,2.362268e-02,3709,0.235294,14.0,0.011687,5,256.0,66,1.0,2.0,54.0
2,Baroque,8203,3,0,3,7673.0,530.0,1.743578e-02,3647,0.250000,16.0,0.017191,1,406.0,34,1.0,2.0,57.0
19,Independent film,4155,20,0,3,3287.0,868.0,1.555586e-03,1978,0.000000,1.0,0.007920,12,920.0,7,1.0,2.0,63.0
32,Lyricist,2758,33,1,2,2717.0,41.0,5.367665e-03,3120,0.000000,1.0,0.004100,31,22.0,136,1.0,2.0,68.0
5,Flute,7595,6,0,3,7106.0,489.0,4.069891e-02,3762,0.368421,22.0,0.008381,8,356.0,44,1.0,2.0,84.0
30,Big band,2876,31,0,3,2554.0,322.0,2.532333e-02,3723,0.000000,1.0,0.005438,22,364.0,42,1.0,2.0,96.0
11,Conducting,5745,12,2,1,5259.0,486.0,5.269886e-02,3780,3.541667,116.0,0.006453,18,294.0,56,1.0,2.0,128.0
33,Gustav Mahler,2654,34,0,3,1897.0,757.0,1.981251e-01,3802,0.210526,12.0,0.003548,37,938.0,6,1.0,2.0,138.0
67,Chord progression,1236,68,1,2,991.0,245.0,1.627354e-02,3614,0.083333,3.0,0.001406,77,150.0,91,1.0,2.0,142.0


In [ ]:
# features_df.dispersion = features_df.dispersion.fillna(0.0)
# features_df.shortest_path_length_from_source = features_df.shortest_path_length_from_source.fillna(-1)

# Basic Plotting

In [ ]:
sns.pairplot(features_df)

In [ ]:
sns.heatmap(features_df.corr())